In [ ]:
import pandas as pd

In [ ]:
dataset_df=pd.read_excel('DataSet.xlsx')

In [ ]:
dataset_df.head(10)

,id,name,host_id,host_name,neighbourhood,neighbourhood_cleansed,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,calculated_host_listings_count_shared_rooms,reviews_per_month
0,312761,Spacious Central London Apartment by Hoxton Sq...,1608226,Pippa,"London, England, United Kingdom",Hackney,51.52704,-0.08129,Entire home/apt,$160.00,9,122,2024-02-03 00:00:00,0,0.86
1,13913,Holiday London DB Room Let-on going,54730,Alina,"Islington, Greater London, United Kingdom",Islington,51.56861,-0.1127,Private room,$65.00,1,41,2023-11-26 00:00:00,0,0.25
2,15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,"London, United Kingdom",Kensington and Chelsea,51.4878,-0.16813,Entire home/apt,$120.00,4,94,2023-05-01 00:00:00,0,0.54
3,159736,A double Room 5mins from King's College Hospital,766056,Trevor,"Lambeth, England, United Kingdom",Lambeth,51.46788,-0.09993,Private room,$65.00,4,96,2023-04-29 00:00:00,0,0.62
4,165336,Charming Flat in Notting Hill,761400,Nathan,"Kensington, England, United Kingdom",Kensington and Chelsea,51.51735,-0.21076,Entire home/apt,$100.00,1,237,2024-03-17 00:00:00,0,1.57
5,313710,Pretty Home Kensal Green NW10 1 bed,1612623,Alex,NaN,Brent,51.53347,-0.23292,Entire home/apt,NaN,2,10,2015-09-13 00:00:00,0,0.07
6,168359,Stunning 2 Bed Apartment in Elephant & Castle,648725,Michele,"Greater London, England, United Kingdom",Southwark,51.49185,-0.08823,Entire home/apt,NaN,2,9,2019-01-02 00:00:00,0,0.06
7,17402,Fab 3-Bed/2 Bath & Wifi: Trendy W1,67564,Liz,"London, Fitzrovia, United Kingdom",Westminster,51.52195,-0.14094,Entire home/apt,$474.00,3,56,2024-02-19 00:00:00,0,0.35
8,24328,"Battersea live/work artist house, garden communal",41759,Joe,"London, United Kingdom",Wandsworth,51.47072,-0.16266,Entire home/apt,NaN,2,94,2022-07-19 00:00:00,0,0.58
9,170524,STUNNING APT IN ST JAMES,194769,D,NaN,Westminster,51.50612,-0.14029,Entire home/apt,"$1,300.00",14,0,NaN,0,NaN


In [ ]:
# remove irrelevant columns

required_columns=['id','name','host_id','neighbourhood_cleansed','latitude','longitude','room_type','price','number_of_reviews','reviews_per_month']
cleaned_data=dataset_df[required_columns]



In [ ]:
cleaned_data.isnull().sum()

id                            0
name                          0
host_id                      15
neighbourhood_cleansed       75
latitude                     41
longitude                   123
room_type                    30
price                     25651
number_of_reviews            17
reviews_per_month          9882
dtype: int64



considering the number of missing values in the Price and reviews_per_month column needs different imputation method to fix the values instead removing the missing row/column from the given dataset.

Future work:

since the number of missing values in the host_id, neighbourhood_cleansed, latitude, longitude are very less and not suitable for imputation and does not have any impact, we are consider removing those missing value rows from the dataset.

Also, missing room_type value would be modified as 'unkown' and number_of_reviews would be modified as 0 for our convenience to avoid data undersampling.

In [ ]:
# remove rows with missing values in critical columns

cleaned_data=cleaned_data.dropna(subset=['host_id','neighbourhood_cleansed','latitude','longitude','room_type','number_of_reviews'])

In [ ]:
cleaned_data.isnull().sum()

id                            0
name                          0
host_id                       0
neighbourhood_cleansed        0
latitude                      0
longitude                     0
room_type                     0
price                     25553
number_of_reviews             0
reviews_per_month          9786
dtype: int64

In [ ]:
# convert price column to string to perform string operation

cleaned_data['price']=cleaned_data['price'].astype(str)


<ipython-input-21-83d3ceae935a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['price']=cleaned_data['price'].astype(str)


In [ ]:
# remove rows with non numerice price values

cleaned_data['price']=cleaned_data['price'].str.replace('$','').str.replace(',','').str.strip()
cleaned_data=cleaned_data[cleaned_data['price'].apply(lambda x: x.replace('.','',1).isdigit())]

<ipython-input-22-dabded106814>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['price']=cleaned_data['price'].str.replace('$','').str.replace(',','').str.strip()


In [ ]:
# convert price to float

cleaned_data['price']=cleaned_data['price'].astype(float)

In [ ]:
# filling missing values in price with median value

median_price=cleaned_data['price'].median()
cleaned_data['price'].fillna(median_price,inplace=True)

In [ ]:
# add non numeric reviews_per_month column with 0

cleaned_data['reviews_per_month']=cleaned_data['reviews_per_month'].astype(str).str.replace('f','0').str.replace('nan','0').str.strip()

cleaned_data['reviews_per_month']=cleaned_data['reviews_per_month'].astype(float)

In [ ]:
# add non_numeric number_of_reviews column woth 0

cleaned_data['number_of_reviews']=cleaned_data['number_of_reviews'].astype(str).str.replace('f','0').str.replace('t','0').str.strip()

cleaned_data['number_of_reviews']=cleaned_data['number_of_reviews'].astype(float)

In [ ]:
dataset_cleaned=cleaned_data.to_excel('DataSet_Cleaned.xlsx')

In [ ]:
cleaned_data.isnull().sum()

id                        0
name                      0
host_id                   0
neighbourhood_cleansed    0
latitude                  0
longitude                 0
room_type                 0
price                     0
number_of_reviews         0
reviews_per_month         0
dtype: int64

In [ ]:
cleaned_data.describe()

,id,price,number_of_reviews,reviews_per_month
count,1.923000e+04,19230.000000,19230.000000,19230.000000
mean,1.196923e+15,185.988376,49.375069,0.832060
std,2.500706e+16,631.276082,75.097002,1.263811
min,1.391300e+04,0.000000,0.000000,0.000000
25%,1.610618e+07,65.000000,6.250000,0.160000
50%,3.032144e+07,120.000000,23.000000,0.470000
75%,4.444340e+07,200.000000,60.000000,1.030000
max,5.421905e+17,53588.000000,1457.000000,75.000000


Data sorting algorithm

In [ ]:
import time

In [ ]:
# function to measure sorting time

def sort_time_measurement(df,column):
  start_time=time.time()
  sorted_df=df.sort_values(by=[column])
  end_time=time.time()
  return end_time-start_time

def main():
  # column to sort by
  column_to_sort=['price','number_of_reviews','reviews_per_month']

  for column in column_to_sort:
    sort_time=sort_time_measurement(cleaned_data,column)
    print(f"sorting by {column} took {sort_time:.6f} seconds")

if __name__=='__main__':
  main()

sorting by price took 0.023213 seconds
sorting by number_of_reviews took 0.008644 seconds
sorting by reviews_per_month took 0.009537 seconds
